[Bag of words](https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles/4855001-representez-votre-corpus-en-bag-of-words)

[Feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html) ; [FE - Count extraction](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.toarray())
print vectorizer.vocabulary_.get('the')


vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names())

print(X2.toarray())


[u'and', u'document', u'first', u'is', u'one', u'second', u'the', u'third', u'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]
6
[u'and this', u'document is', u'first document', u'is the', u'is this', u'second document', u'the first', u'the second', u'the third', u'third one', u'this document', u'this is', u'this the']
[[0 0 1 1 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 1]]


[Stemming, stopword... & NLTK](https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles/4854971-nettoyez-et-normalisez-les-donnees)

[Wiki - Racinisation](https://fr.wikipedia.org/wiki/Racinisation)

Afin de faciliter les choses, on stem avant de bager (Raciniser/Suppr. des stopwords avant de grouper)

[TF-IDF](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76)

In [7]:
#A activer afin de récupèrer les corpus de NLTK
#Selectionner 'all-corpora'
#import nltk
#nltk.download()

from nltk.corpus import stopwords
sw = stopwords.words("english")
print len(sw)

179


In [13]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
print stemmer.stem("responsiveness")#Permet d'obtenir le radical d'un mot (voir wiki)
print stemmer.stem("responsivity")
print stemmer.stem("unresponsive")

respons
respons
unrespons
